In [1]:
import pandas as pd

In [9]:
def read_train_val_test(train_path, val_path, test_path):
    train_data = pd.read_csv(train_path, index_col=0)
    val_data = pd.read_csv(val_path, index_col=0)
    test_data = pd.read_csv(test_path, index_col=0)
    return train_data, val_data, test_data



In [10]:
senate_train_data, senate_val_data, senate_test_data = read_train_val_test("train_data/merged_senate_districts_2011_2012.csv", "valid_data/merged_senate_districts_2013_2014.csv", "test_data/merged_senate_districts_2015_2016.csv")


(48, 354) (82, 354) (85, 364)
                                           Estimate; SEX AND AGE - Total population  \
Berkshire, Hampshire & Franklin                                            149077.0   
Berkshire, Hampshire, Franklin & Hampden                                   155357.0   
Bristol & Norfolk                                                          158442.5   
Cape & Islands                                                             165409.5   
Fifth Middlesex                                                            168316.0   
First Bristol & Plymouth                                                   163327.0   
First Essex                                                                170080.5   
First Essex & Middlesex                                                    169923.0   
First Hampden & Hampshire                                                  160021.5   
First Middlesex                                                            164765.5   
First Middles

In [21]:
def clean_data(train_data, val_data, test_data):
    # make all text binary
    train_data = train_data.drop("name", axis=1)
    train_data['sex'].replace('f', 1, inplace=True)
    train_data['sex'].replace('m', 0, inplace=True)
    train_data['party'].replace('Democratic', 1, inplace=True)
    train_data['party'].replace('Republican', 0, inplace=True)
    
    # fill NaN's with mean from column
    train_data['sex'] = train_data['sex'].fillna(train_data['sex'].mean())
    train_data['sex'] = train_data['sex'].fillna(train_data['sex'].mean())
    print(train_data)
    return cleaned_train, cleaned_val, cleaned_test

In [22]:
cleaned_train, cleaned_val, cleaned_test = clean_data(senate_train_data, senate_val_data, senate_test_data)

                                           Estimate; SEX AND AGE - Total population  \
Berkshire, Hampshire & Franklin                                            149077.0   
Berkshire, Hampshire, Franklin & Hampden                                   155357.0   
Bristol & Norfolk                                                          158442.5   
Cape & Islands                                                             165409.5   
Fifth Middlesex                                                            168316.0   
First Bristol & Plymouth                                                   163327.0   
First Essex                                                                170080.5   
First Essex & Middlesex                                                    169923.0   
First Hampden & Hampshire                                                  160021.5   
First Middlesex                                                            164765.5   
First Middlesex & Norfolk                  

NameError: name 'cleaned_train' is not defined